# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import numpy as np
import pandas as pd
import os
from datetime import date
from IPython.core.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline
DATA_FOLDER = 'Data'


## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### A) Pre-processing data

In [ ]:
GUINEA_PATH  = DATA_FOLDER + '/ebola/guinea_data/'
LIBERIA_PATH = DATA_FOLDER + '/ebola/liberia_data/'
SL_PATH      = DATA_FOLDER + '/ebola/sl_data/'

"""Read data"""

GUINEA_DATAS  = [pd.read_csv(GUINEA_PATH  + file) for file in os.listdir( GUINEA_PATH ) if file.endswith(".csv")]
LIBERIA_DATAS = [pd.read_csv(LIBERIA_PATH + file) for file in os.listdir( LIBERIA_PATH )if file.endswith(".csv")]
SL_DATAS      = [pd.read_csv(SL_PATH      + file) for file in os.listdir( SL_PATH )     if file.endswith(".csv")]

In [ ]:
"""Concatenate all files of each country and replace missing value by zero"""

GUINEA_dframes  = pd.concat(GUINEA_DATAS)
LIBERIA_dframes = pd.concat(LIBERIA_DATAS)
SL_dframes      = pd.concat(SL_DATAS)

GUINEA_dframes  = GUINEA_dframes .fillna(0)
LIBERIA_dframes = LIBERIA_dframes.fillna(0)
SL_dframes      = SL_dframes     .fillna(0)

In [ ]:
"""First of all, take a look at data header and decide which columns are useful"""

GUINEA_dframes  = GUINEA_dframes [['Date', 'Description', 'Totals']]
LIBERIA_dframes = LIBERIA_dframes[['Date', 'Variable', 'National']]
SL_dframes      = SL_dframes     [['date', 'variable', 'National']]

In [ ]:
"""Rename for having the same format"""

LIBERIA_dframes = LIBERIA_dframes.rename(index=str, columns={"Variable": "Description", "National": "Totals"})
SL_dframes      = SL_dframes.     rename(index=str, columns={"date": "Date","variable": "Description", "National": "Totals"})

In [ ]:
"""Change format Date of data provided by LIBERIA"""
"""Change 6/16/2014 (MM/DD/YYYY) to 2014-06-16 (YYYY-MM-DD)"""
"""Attention with data '6/16/17' 17 means 2017 :)"""

def changeDateFormat(date):
    string = date.split('/')
    if (len(string[0]) == 1) : string[0] = '0' + string[0]
    if (len(string[1]) == 1) : string[1] = '0' + string[1]
    if (len(string[2]) == 2) : string[2] = '20'+ string[2]
    return string[2] + '-' + string[0] + '-' + string[1]

LIBERIA_dframes['Date'] = LIBERIA_dframes['Date'].apply(lambda x : changeDateFormat(str(x)))

In [ ]:
"""Parsing string Date to object TimeDate"""

def parseDate(string):
    result = pd.to_datetime(string, format='%Y-%m-%d', errors='coerce')
    if (type(result) is pd.tslib.NaTType):
        print('Problem man !!!')
    return result

GUINEA_dframes ['Date'] = GUINEA_dframes ['Date'].apply(lambda x : parseDate(str(x)))
LIBERIA_dframes['Date'] = LIBERIA_dframes['Date'].apply(lambda x : parseDate(str(x)))
SL_dframes     ['Date'] = SL_dframes     ['Date'].apply(lambda x : parseDate(str(x)))

In [ ]:
"""Insert an extra column Country"""

GUINEA_dframes .insert(3, 'Country', 'GUINEA')
LIBERIA_dframes.insert(3, 'Country', 'LIBERIA')
SL_dframes     .insert(3, 'Country', 'SL')

In [ ]:
"""Set indexes"""

GUINEA_dframes.index  = range(len(GUINEA_dframes))
LIBERIA_dframes.index = range(len(LIBERIA_dframes))
SL_dframes.index      = range(len(SL_dframes))


In [ ]:
"""Drop all non-integer elements in column 'Totals'"""

GUINEA_dframes  = GUINEA_dframes[ GUINEA_dframes  ['Totals'].apply(lambda x: str(x).isdigit())]
#LIBERIA_dframes = LIBERIA_dframes[ LIBERIA_dframes['Totals'].apply(lambda x: x.isdigit() or str(x).isdigit())]
SL_dframes      = SL_dframes[ SL_dframes          ['Totals'].apply(lambda x: str(x).isdigit())]

In [ ]:
"""Convert all integer of type string into integer"""

GUINEA_dframes ['Totals'] = pd.to_numeric( GUINEA_dframes ['Totals'], errors='coerce')
LIBERIA_dframes['Totals'] = pd.to_numeric( LIBERIA_dframes['Totals'], errors='coerce')
SL_dframes     ['Totals'] = pd.to_numeric( SL_dframes     ['Totals'], errors='coerce')

### B) Computation

#### GUINEA
In reports of GUINEA, there are 2 types of new cases: 
* 'Total new cases' (with a litle 'n' in new cases).
* 'New cases' (with a capital 'N' in New cases) which is relevant because we can take into account the case 'New cases of confirmed..' too.



How do I compute the average ? Well, for each month, I count the total 'New cases' and then devided it by the number of days reported in this month. Par example, in August we have the total of 'New cases' is 300 and it is reported in 3 days. Then we have the average is 300/3 = 100.

In [ ]:
print('The daily average per month of new cases')

"""Find key New cases"""
GUINEA_newcases = GUINEA_dframes[ [('New cases' in desc) for desc in GUINEA_dframes.Description]]

"""Groupby Month for being able to aggregate in Totals"""
daily = GUINEA_newcases.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a month"""
days = GUINEA_newcases.groupby(GUINEA_newcases.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'new cases' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))


For me, a confirmation of a death is valided. So I count only the cases that have the key 'confirmed'

In [ ]:
print('The daily average per month of deaths')

GUINEA_deaths = GUINEA_dframes[ [('confirmed' in desc and 
                                  'deaths'    in desc and 
                                  'Total deaths (confirmed + probables + suspects)' not in desc or    
                                  (   'deaths registered' in desc and
                                      '(confirmed)'   not in desc and
                                      '(probables)'   not in desc and
                                      '(suspects)'    not in desc 
                                  )
                                ) for desc in GUINEA_dframes.Description]]

"""Groupby Month for being able to aggregate in Totals"""
daily = GUINEA_deaths.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a month"""
days = GUINEA_deaths.groupby(GUINEA_deaths.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'deaths' of month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

#### LIBERIA

The same idea here, in the data we have 4 different words: *New cases*, *New case*, *New Case*, *New Cases*. So we just need to collect all new cases and compute

In [ ]:
""""""
print('The daily average per month of deaths')


LIBERIA_newcases = LIBERIA_dframes[ [('New cases' in desc  or 
                                      'New case'  in desc  or
                                      'New Case'  in desc  or
                                      'New Cases' in desc 
                                     ) for desc in LIBERIA_dframes.Description]]


"""Groupby Month for being able to aggregate in Totals"""
daily = LIBERIA_newcases.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a specific month"""
days = LIBERIA_newcases.groupby(LIBERIA_newcases.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'new cases' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

In [ ]:
LIBERIA_deaths = LIBERIA_dframes[ [( 'deaths' in desc  or 
                                     'death'  in desc  and                                     
                                       'Total death/s in confirmed cases' not in desc and
                                    'Total death/s in probable cases' not in desc and
                                    'Total death/s in suspected cases' not in desc 
                                   ) for desc in LIBERIA_dframes.Description]]

"""Groupby Month for being able to aggregate in Totals"""
daily = LIBERIA_deaths.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a specific month"""
days = LIBERIA_deaths.groupby(LIBERIA_deaths.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'deaths' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

#### SL

In [ ]:
print('The daily average per month of deaths')


SL_newcases = SL_dframes[ [('new_' in desc and 'deaths' not in desc and
                            'new_negative' not in desc
                                     ) for desc in SL_dframes.Description]]
"""Groupby Month for being able to aggregate in Totals"""
daily = SL_newcases.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a specific month"""
days = SL_newcases.groupby(SL_newcases.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]


"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'new cases' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

In [ ]:
SL_deaths = SL_dframes[ [('deaths' in desc or 'Deaths' in desc
                                     ) for desc in SL_dframes.Description]]


"""Groupby Month for being able to aggregate in Totals"""
daily = SL_deaths.set_index(['Date']).groupby(pd.TimeGrouper('M'))

"""Groupby Date for being able to count number of days reported in a specific month"""
days = SL_deaths.groupby(SL_deaths.Date)
arr = np.array(list(days.groups.keys()))
arr = [x.month for x in arr]

"""Sum over month divided by number of days (length of keys)"""

timestamps = list(daily.groups.keys())
sums = daily.Totals.sum()
nbMonths = len(timestamps)
for i in range(nbMonths):
    month = timestamps[i].month
    print("The average 'deaths' on month {m} is {v}"
              .format(m=month, v=sums[i]/arr.count(month)))

#### Merge 3 frames

In [ ]:
df_all            = pd.concat([GUINEA_dframes, LIBERIA_dframes, SL_dframes])
df_all.index      = range(len(df_all))
df_all

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
PATH  = DATA_FOLDER + '/microbiome/'

In [ ]:
""" Read excel files"""

files = [pd.ExcelFile(PATH  + file) for file in os.listdir(PATH) if file.startswith("MID")]
frames = [file.parse('Sheet 1') for file in files]

In [ ]:
"""Store the end possition of each sheet """
indexes = [len(frame) for frame in frames]

In [ ]:
"""Single dataframe with 9 spreadsheets"""
for frame in frames:
    frame.columns = ['TAXON', 'COUNT']
micro_frames = pd.concat(frames)

In [ ]:
"""Import the 10th file"""

metadata = [pd.ExcelFile(PATH  + file) for file in os.listdir(PATH) if file.startswith("meta")]
metadata_frame = [ frame.parse("Sheet1") for frame in metadata]

In [ ]:
"""Create 3 numpy columns SAMPLE, BARCODE, GROUP"""
barcode = np.array([])
group = np.array([])
sample = np.array([])
for index, row in metadata_frame[0].iterrows():
    endPos = indexes[index]
    if pd.isnull(row['SAMPLE']): 
        sample = np.append(sample, np.array(['unknown']*endPos))
    else :
        sample = np.append(sample, np.array([row['SAMPLE']]*endPos))
    barcode = np.append(barcode, np.array([row['BARCODE']]*endPos))
    group   = np.append(group, np.array([row['GROUP']]*endPos))


In [ ]:
"""Concatenation all columns"""

micro_frames['SAMPLE'] = sample
micro_frames['BARCODE'] = barcode
micro_frames['GROUP'] = group

In [ ]:
"""reorder columns AND groupby..."""
micro_frames = micro_frames[['BARCODE', 'TAXON', 'GROUP', 'COUNT','SAMPLE']]
micro_frames.set_index(['BARCODE', 'TAXON'])

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
import random
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

In [ ]:
data_titanic = pd.read_excel(DATA_FOLDER + '/titanic.xls')
data_titanic

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
data_titanic['nb_people'] = 1
types = data_titanic.dtypes

# pclass

In [ ]:
pclass = data_titanic['pclass'].dropna()
print("the type of pclass is: %s"%(types[0]))
print("There are %d different classes: "%(len(set(pclass))))
pclass.unique()

In [ ]:
# survived

In [ ]:
survived = data_titanic['survived'].dropna()
print("the type of survived is: %s"%(types[1]))
print("There are %d possible values for survived :"%(len(set(survived))))
survived.unique()

In [ ]:
# name

In [ ]:
name = data_titanic['name'].dropna()
print("the type of name is: %s"%(types[2]))
print("There are %d possible values for name :"%(len(set(name))))
name.unique()

In [ ]:
# sex

In [ ]:
sex = data_titanic['sex'].dropna()
print("the type of sex is: %s"%(types[3]))
print("There are %d possible values for sex :"%(len(set(sex))))
sex.unique()

In [ ]:
# age

In [ ]:
age = data_titanic['age'].dropna()
print("the type of age is: %s"%(types[4]))
print("the min age is %f and the max age is %f"%(age.min(),age.max()))
print("There are %d possible values for age :"%(len(set(age))))
age.unique()

In [ ]:
# sibsp

In [ ]:
sibsp = data_titanic['sibsp'].dropna()
print("the type of sibsp is: %s"%(types[5]))
print("There are %d possible values for sibsp :"%(len(set(sibsp))))
sibsp.unique()

In [ ]:
# parch

In [ ]:
parch = data_titanic['parch'].dropna()
print("the type of parch is: %s"%(types[6]))
print("There are %d possible values for parch :"%(len(set(parch))))
parch.unique()

In [ ]:
# ticket

In [ ]:
ticket = data_titanic['ticket'].dropna()
print("the type of ticket is: %s"%(types[7]))
print("There are %d possible values for ticket :"%(len(set(ticket))))
ticket.unique()

In [ ]:
# fare

In [ ]:
fare = data_titanic['fare'].dropna()
print("the type of fare is: %s"%(types[8]))
print("the min fare is %f and the max fare is %f"%(fare.min(),fare.max()))
print("There are %d possible values for fare :"%(len(set(fare))))
fare.unique()

In [ ]:
# cabin

In [ ]:
cabin = data_titanic['cabin'].dropna()
print("the type of cabin is: %s"%(types[9]))
print("There are %d possible values for cabin :"%(len(set(cabin))))
cabin.unique()

In [ ]:
# embarked

In [ ]:
embarked = data_titanic['embarked'].dropna()
print("the type of embarked is: %s"%(types[10]))
print("There are %d possible values for embarked :"%(len(set(embarked))))
embarked.unique()

In [ ]:
# boat

In [ ]:
boat = data_titanic['boat'].dropna()
print("the type of boat is: %s"%(types[11]))
print("There are %d possible values for boat :"%(len(set(boat))))
boat.unique()

In [ ]:
# body

In [ ]:
body = data_titanic['body'].dropna()
print("the type of boat is: %s"%(types[12]))
print("There are %d body values for body :"%(len(set(body))))
body.unique()

In [ ]:
# home.dest

In [ ]:
home_dest = data_titanic['home.dest'].dropna()
print("the type of home.dest is: %s"%(types[13]))
print("There are %d body values for home.dest :"%(len(set(home_dest))))
home_dest.unique()

In [ ]:
We can transform the following attributes in categorical:
-  sex 
-  survived 
-  pclass 
-  embarked
-  sibsp
-  parch

In [ ]:
pd.Categorical(data_titanic.sex)
pd.Categorical(data_titanic.survived)
pd.Categorical(data_titanic.embarked)
pd.Categorical(data_titanic.pclass)
pd.Categorical(data_titanic.sibsp)
pd.Categorical(data_titanic.parch)

In [ ]:
# Travel class

In [ ]:
#count the number of passangers per class and represent it in a histogram
plot_class = data_titanic.pclass.value_counts().plot(kind='bar', title='Number of passangers per travel class')
plot_class.set_xlabel("class")
plot_class.set_ylabel("nb of passangers")

In [ ]:
# Embarkation port

In [ ]:
#add a random embarkation port when it is missing
data_titanic.embarked.fillna(random.choice(['S','C','Q']))
plot_port = data_titanic.embarked.value_counts().plot(kind='bar', title='Number of passangers per embarkation port')
plot_port.set_xlabel("port")
plot_port.set_ylabel("nb of passangers")

In [ ]:
# Sex

In [ ]:
#count the number of people of each sex
plot_sex = data_titanic.sex.value_counts().plot(kind='bar', title='')
plot_sex.set_xlabel("sex")
plot_sex.set_ylabel("nb of passangers")

In [ ]:
# Age

In [ ]:
#drop all the fields with missing age and associate every age with the decade interval it is in
a = pd.cut(data_titanic.age.dropna(), [0,10,20,30,40,50,60,70,80])
b = pd.DataFrame(a).reset_index()
c = b.groupby('age').count()
c.plot(kind="bar")



In [ ]:
# I assume we only have the following floors and I don't take into consideration the fields that don't have a 
# valid cabin number
floors = ['A','B','C','D','E','F','G']
cabins = data_titanic.cabin.dropna()
data_titanic['floor']= data_titanic['cabin'].apply(lambda x: str(x)[0] if str(x)[0] in floors else 'Unknown')

floor_groups = data_titanic[['floor', 'nb_people']].groupby(['floor']).size()
series_floors = pd.Series(floor_groups, index=['A','B','C','D','E','F','G','Unknown'], name='')
series_floors.plot.pie(figsize=(6, 6), title = 'Nb of people per floor')



In [ ]:
#calculate the number of people per travel class
nb_people_1 = data_titanic[pclass == 1]['pclass'].count()
nb_people_2 = data_titanic[pclass == 2]['pclass'].count()
nb_people_3 = data_titanic[pclass == 3]['pclass'].count()

#calculate the number of people that survived and group it by class
survived_class = data_titanic[data_titanic.survived == 1][['pclass','nb_people']].groupby(['pclass']).size()
#normalise the nb of people that survived by class
series_class1 = pd.Series([survived_class[1]/nb_people_1, 1- survived_class[1]/nb_people_1], name ='', index = ["survived", "dead"])
series_class2 = pd.Series([survived_class[2]/nb_people_2, 1- survived_class[2]/nb_people_2],name = '',index = ["survived", "dead"])                          
series_class3 = pd.Series([survived_class[3]/nb_people_3,1 -survived_class[3]/nb_people_3 ], name = '',index = ["survived", "dead"])



In [ ]:
series_class1.plot.pie(figsize=(6, 6), title = 'Nb of people survived in 1st class')

In [ ]:
series_class2.plot.pie(figsize=(6, 6), title = 'Nb of people survived in 2nd class')

In [ ]:
series_class3.plot.pie(figsize=(6, 6), title = 'Nb of people survived in 3rd class')

In [ ]:

s_sex_class = data_titanic[data_titanic.survived == 1][['pclass','sex','nb_people']].groupby(['pclass','sex'])['pclass'].count().unstack('sex')
s_sex_class = s_sex_class/data_titanic[data_titanic.survived== 1]['nb_people'].count()

plot_s = s_sex_class.plot(kind='bar', stacked=True)
plot_s.set_xlabel('Class')
plot_s.set_ylabel('nb of people')
plot_s.set_title('Survived passenger by class and sex')
s_sex_class

In [ ]:
age = data_titanic['age'].dropna()
#divide people by age in two intervals: younger than 28 and older than 28
age_cut = pd.cut(age, [0,28,80], 
                  labels=['<28', '>=28'], include_lowest=True)
#add a new field age-cat
data_titanic['age-cat'] = age_cut
catsdata = data_titanic[['age-cat']]
data_titanic[['age-cat', 'nb_people']].groupby(['age-cat']).sum()

In [ ]:
survived = data_titanic[data_titanic.survived == 1][['pclass','nb_people','sex','age-cat']].groupby(['age-cat','pclass','sex']).sum()
survived